In [1]:
from scipy.spatial.transform import Rotation as R
import open3d as o3d
import numpy as np

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## bin to xyz 

In [1]:
import struct
#file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000.bin'
file_path = 'D:\VS2022Projects/arap-volume-tracking-main\data\Levi\Frame_00vg_512_000.bin'
print(file_path)

def load_bin(file_path):
    with open(file_path, 'rb') as f:
        # Read the first 4 bytes as an integer (this is the count 'c')
        c = struct.unpack('i', f.read(4))[0]
        
        # Initialize an empty numpy array to store the vectors
        vectors = np.zeros((c, 3), dtype=np.float32)
        
        for i in range(c):
            # Read the next 12 bytes (3 floats) for the Vector3
            x, y, z = struct.unpack('3f', f.read(12))
            vectors[i] = (x, y, z)
    
    return vectors

centers = load_bin(file_path)
print(centers, centers.__len__())

D:\VS2022Projects/arap-volume-tracking-main\data\Levi\Frame_00vg_512_000.bin
[[ 7.1746481e-43  2.3681944e-43 -2.6019400e-01]
 [-4.0894400e-02 -3.1880599e-01  3.5308094e-03]
 [ 1.7312426e-39  9.2194229e-41 -2.7755576e-17]
 [ 2.3509887e-38  8.4077908e-45  5.7397185e-41]
 [ 9.1871369e-40 -1.6940659e-21  2.3509887e-38]
 [ 1.5414283e-44  5.6679720e-41  8.2687820e-40]
 [-1.0842022e-19  2.3509887e-38  7.0064923e-45]
 [ 5.8832115e-41  2.7586522e-40 -8.3886080e+06]
 [ 2.3510789e-38  8.4077908e-45  5.7397185e-41]
 [ 1.0105492e-39 -6.7762636e-21  2.3509887e-38]
 [ 1.6815582e-44  5.6320988e-41  1.1023847e-39]
 [-1.6940659e-21  2.3509887e-38  1.8216880e-44]
 [ 5.5962255e-41  1.1942202e-39 -1.6940659e-21]
 [ 2.3509887e-38  1.6815582e-44  5.6320988e-41]
 [ 1.0105492e-39 -6.7762636e-21  2.3509887e-38]
 [ 1.4012985e-44  5.7397185e-41  1.1942202e-39]
 [-1.6940659e-21  2.3509887e-38  1.9618179e-44]
 [ 5.5962255e-41  1.2860557e-39 -2.6469780e-23]
 [ 2.3509887e-38  2.2420775e-44  5.4886058e-41]
 [ 1.561562

In [4]:
file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000_out.bin'
centers = load_bin(file_path)
print(centers, centers.__len__())

[[ 0.30941448  0.9935851  -0.10177808]
 [-0.01174076  0.28923604  0.2646991 ]
 [-0.3199594   0.37756237 -0.21296251]
 ...
 [ 0.03638093  1.5659114  -0.29961684]
 [ 0.00987653  0.13647208  0.1315317 ]
 [ 0.33245873  0.64176136 -0.19910005]] 1000


In [42]:
print(centers[4])

[ 0.20597723  1.8731184  -0.02225684]


## xyz to bin

In [2]:
import struct
import numpy as np

def save_bin(file_path, centers):
    # Open the file in write-binary mode
    with open(file_path, 'wb') as f:
        # Write the number of vectors as a 4-byte integer
        c = len(centers)
        f.write(struct.pack('i', c))
        
        # Write each vector (x, y, z) as three 4-byte floats
        for vector in centers:
            f.write(struct.pack('3f', *vector))

# Example usage
new_file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test\Data\pent\centers\mesh_000res_1000_000_out.bin'

# Save the centers array back to a bin file
save_bin(new_file_path, centers)

## draw centers

In [3]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.05  # Adjust the radius as needed

# Create a mesh sphere for each point and translate it to the point's location
for point in centers:
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    
    # Add the sphere to the list
    spheres.append(mesh_sphere)

# Combine all the spheres into one mesh
all_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    all_spheres += sphere

# Optionally, you can set the color of the spheres
all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color

# Draw the combined mesh
o3d.visualization.draw_geometries([all_spheres])

## Apply Transformation

In [43]:
#file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformations_1.txt'
file_path = r'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi\transformations.txt'

# Initialize a list to store the parsed data
real_data = []
dual_data = []

# Open the file and read each line
with open(file_path, 'r') as file:
    for line in file:
        # Strip any leading/trailing whitespace and split the line by semicolon
        values = line.strip().split(';')
        
        # Convert the string values to floats
        real_values = tuple(float(value) for value in values[:4])  # First four as Real
        dual_values = tuple(float(value) for value in values[4:])  # Last four as Dual
        
        # Append the Real and Dual parts to their respective lists
        real_data.append(real_values)
        dual_data.append(dual_values)

def create_matrix_3x3(real):
    # Unpack the real and dual components
    rx, ry, rz, rw = real
    
    # Initialize the matrix
    matrix = np.zeros((3, 3))
    
    # Rotation part of the matrix
    matrix[0, 0] = rw * rw + rx * rx - ry * ry - rz * rz
    matrix[0, 1] = 2 * rx * ry - 2 * rw * rz
    matrix[0, 2] = 2 * rx * rz + 2 * rw * ry
    matrix[1, 0] = 2 * rx * ry + 2 * rw * rz
    matrix[1, 1] = rw * rw - rx * rx + ry * ry - rz * rz
    matrix[1, 2] = 2 * ry * rz - 2 * rw * rx
    matrix[2, 0] = 2 * rx * rz - 2 * rw * ry
    matrix[2, 1] = 2 * ry * rz + 2 * rw * rx
    matrix[2, 2] = rw * rw - rx * rx - ry * ry + rz * rz
    
    
    return matrix

print(real_data.__len__())

2000


In [46]:
centers_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/centers/Frame_00res_2000_000.xyz'
centers = np.loadtxt(centers_path)

In [2]:
def quaternion_multiply(quaternion1, quaternion0):
    x0, y0, z0, w0 = quaternion0
    x1, y1, z1, w1 = quaternion1
    return np.array([-x1*x0 - y1*y0 - z1*z0 + w1*w0,
                         x1*w0 + y1*z0 - z1*y0 + w1*x0,
                        -x1*z0 + y1*w0 + z1*x0 + w1*y0,
                         x1*y0 - y1*x0 + z1*w0 + w1*z0], dtype=np.float64)

In [48]:
from scipy.spatial.transform import Rotation as R
transformed_centers = []
Transformations = []
for i in range(len(centers)):
    real_values = real_data[i]  # Replace with actual Real values
    dual_values = dual_data[i]  # Replace with actual Dual values
    r = R.from_quat(real_data[i])
    Transformations.append(r.as_matrix())
    point = centers[i]
    x1 = real_values[0]
    y1 = real_values[1]
    z1 = real_values[2]
    w1 = real_values[3]
    x2 = dual_values[0]
    y2 = dual_values[1]
    z2 = dual_values[2]
    w2 = dual_values[3]
    Conjugate = (-x1,-y1,-z1, w1)
    TQ = quaternion_multiply(dual_values, Conjugate)
    Vector = (TQ[1]*2, TQ[2]*2, TQ[3]*2) # not TQ[0]*2, TQ[1]*2, TQ[2]*2, TQ[0] is w 
    transformed_center = Vector +  np.dot(r.as_matrix(), point)
    print(transformed_center)
    transformed_centers.append(transformed_center)
    
#file_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'
#np.savetxt(file_path,transformed_centers)

[0.13163824 1.3015468  0.17161079]
[ 0.02526516  0.9471354  -0.11976492]
[ 0.13601176  0.5687423  -0.16539569]
[-0.13389757  0.8575752   0.02801788]
[ 0.06912288  1.1568489  -0.09975089]
[ 0.10629752  0.5505458  -0.10851371]
[ 0.09302792  1.0068346  -0.17939284]
[-0.07491355  1.3787477  -0.07812908]
[-0.04983017  1.5326016  -0.14237334]
[ 0.05610046  0.91823053 -0.21310009]
[ 0.11534841  0.9744478  -0.13601534]
[-0.10343831  1.4133056  -0.05003477]
[ 0.00371992  1.4862635  -0.12397028]
[-0.03899693  0.73335457  0.08505137]
[-0.08480953  1.1826131  -0.11504915]
[ 0.01480906  1.4341424  -0.19526026]
[-0.20037064  0.12234846  0.05656539]
[ 0.27960682  0.00980967 -0.14778689]
[ 0.1521821   0.3634478  -0.21390897]
[-0.16509987  0.2657988   0.025121  ]
[-0.01650709  1.4179869  -0.1977611 ]
[-0.2043104   0.1945678   0.00976186]
[ 0.06199331  1.3479271  -0.16062236]
[-0.1476641   0.90171933 -0.07114503]
[-0.04028893  0.9285512  -0.08679514]
[-0.15396583  0.01957848  0.2039176 ]
[-0.03623113  1

In [45]:
transformed_centers

[array([ 0.01115989,  0.0556027 , -0.26280662]),
 array([-0.03762572, -0.30548749,  0.00374323]),
 array([ 0.00068964,  0.009307  , -0.00200449]),
 array([ 0.00261432,  0.0123659 , -0.00099915]),
 array([0.00506508, 0.0161194 , 0.00364106]),
 array([ 0.00141362,  0.00899   , -0.00202581]),
 array([0.00432459, 0.0143694 , 0.00350951]),
 array([ 2.59641604e-03,  1.45683000e-02, -8.38860800e+06]),
 array([ 0.00231606,  0.0124716 , -0.00472646]),
 array([ 0.00390782,  0.01353433, -0.00071482]),
 array([0.00195718, 0.01271833, 0.00193603]),
 array([ 0.00113441,  0.0148402 , -0.00521733]),
 array([ 0.00312366,  0.0144009 , -0.00202101]),
 array([0.00350388, 0.01229131, 0.00020678]),
 array([ 0.00490239,  0.0145681 , -0.00244072]),
 array([ 6.27148384e-03,  1.30390003e-02, -7.33199995e-05]),
 array([0.0005101 , 0.00295465, 0.00245014]),
 array([ 0.00043637, -0.0001398 , -0.00017727]),
 array([ 0.0015449 ,  0.00601612, -0.00117542]),
 array([0.00120618, 0.00613537, 0.00390544]),
 array([ 0.006

In [41]:
for i in range(len(centers)):
    if not (np.array_equal(transformed_centers[i], centers[i])):
        print(transformed_centers[i] - centers[i])
        #print(Transformations[i])

[ 0.01115989  0.0556027  -0.00261262]
[0.00326868 0.0133185  0.00021242]
[ 0.00068964  0.009307   -0.00200449]
[ 0.00261432  0.0123659  -0.00099915]
[0.00506508 0.0161194  0.00364106]
[ 0.00141362  0.00899    -0.00202581]
[0.00432459 0.0143694  0.00350951]
[ 0.00259642  0.0145683  -0.00345517]
[ 0.00231606  0.0124716  -0.00472646]
[ 0.00390782  0.01353433 -0.00071482]
[0.00195718 0.01271833 0.00193603]
[ 0.00113441  0.0148402  -0.00521733]
[ 0.00312366  0.0144009  -0.00202101]
[0.00350388 0.01229131 0.00020678]
[ 0.00490239  0.0145681  -0.00244072]
[ 6.27148384e-03  1.30390003e-02 -7.33199995e-05]
[0.0005101  0.00295465 0.00245014]
[ 0.00043637 -0.0001398  -0.00017727]
[ 0.0015449   0.00601612 -0.00117542]
[0.00120618 0.00613537 0.00390544]
[ 0.00689005  0.0131157  -0.00141276]
[0.00111108 0.00464113 0.00368746]
[0.00615215 0.0155844  0.00398147]
[ 0.00223799  0.01319068 -0.00158454]
[ 0.00262926  0.0131763  -0.00069017]
[ 0.00022802 -0.00049979  0.00080135]
[ 0.00246174  0.0117197  -0

In [42]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.05  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(transformed_centers.__len__()):
    point = transformed_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    if not (np.array_equal(point, centers[i])):
        # Color the sphere red if its position is not in reference_centers
        mesh_sphere.paint_uniform_color([1.0, 0.0, 0.0])  # Red color
        changed_sphere += mesh_sphere
        count = count + 1 
    else:
        # Otherwise, color it light grey
        mesh_sphere.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres += sphere

# Optionally, you can set the color of the spheres
#all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color
#mesh = o3d.io.read_triangle_mesh('D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\output\pent/matrix_1/000000.obj')
#mesh.compute_vertex_normals()
# Draw the combined mesh
o3d.visualization.draw_geometries([post_spheres])

160 160


## Use transformed centers and centers to get indices and transformation dual quaternions

In [76]:
transformed_centers_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\pent/transformed_centers_0.txt'

loaded_transformed_centers = np.loadtxt(transformed_centers_path)
print(loaded_transformed_centers)

[[ 0.30941448  0.99358511 -0.10177808]
 [-0.01174076  0.28923604  0.2646991 ]
 [-0.3199594   0.37756237 -0.21296251]
 ...
 [ 0.67638053  2.10591121 -0.45961684]
 [ 0.00987653  0.13647208  0.1315317 ]
 [ 0.33245873  0.64176136 -0.19910005]]


In [2]:
def get_dual_quaternions(original_centers, transformed_centers):
    moved_indices = []
    for i in range(len(original_centers)):
        if not (np.array_equal(original_centers[i], transformed_centers[i])):
            moved_indices.append(i)
    dual_quaternions = np.zeros((len(original_centers), 8), dtype=np.float32)
    for i in moved_indices:
        original = original_centers[i]
        transformed = loaded_transformed_centers[i]
        # Compute rotation - assume for simplicity no rotation (identity quaternion)
        rotation_quaternion = R.from_quat([0, 0, 0, 1])  # Identity rotation
    
        # Compute translation as a vector
        translation = transformed - original
        
        # Convert rotation to a quaternion
        rotation_quat = rotation_quaternion.as_quat()
        
        # Dual quaternion: q + ϵ * (0.5 * t * q)
        translation_quat = np.hstack((translation, [0]))
        dual_quat = np.hstack((rotation_quat, 0.5 * translation_quat))
        dual_quaternions[i] = dual_quat
    return moved_indices, dual_quaternions

In [109]:
indices, dual_quaternions = get_dual_quaternions(centers, transformed_centers)
dual_quaternions

array([[ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       ...,
       [ 0.       ,  0.       ,  0.       , ...,  0.2699999, -0.08     ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ],
       [ 0.       ,  0.       ,  0.       , ...,  0.       ,  0.       ,
         0.       ]], dtype=float32)

In [31]:
#transformed_centers_path = 'D:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/reference_centers_aligned.xyz'
#transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/centers/Frame_00res_4000_001.xyz'
transformed_centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/reference_centers_aligned.xyz'
loaded_transformed_centers = np.loadtxt(transformed_centers_path)
#print(loaded_transformed_centers)

#centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/Frame_00res_4000_000.xyz'
centers_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/centers/dancer_fr0res_2000_005.xyz'
loaded_centers = np.loadtxt(centers_path)
#print(loaded_centers)

indices, dual_quaternions = get_dual_quaternions(loaded_centers, loaded_transformed_centers)

In [33]:
#indices_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Levi/indices.txt'
indices_path = 'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer/indices.txt'
np.savetxt(indices_path, indices, fmt='%d')

In [34]:
dual_quaternions_path = r'G:\VS2022Projects/tvm-editing-master\TVMEditor.Test/bin\Release/net5.0\Data\Dancer\transformations.txt'

# Open the file for writing
with open(dual_quaternions_path, 'w') as file:
    for dq in dual_quaternions:
        # Format the dual quaternion into the desired string format
        dq_str = f"{dq[0]};{dq[1]};{dq[2]};{dq[3]};{dq[4]};{dq[5]};{dq[6]};{dq[7]}"
        file.write(dq_str + '\n')

# Optionally, print the first few lines to verify the saved file
with open(dual_quaternions_path, 'r') as file:
    lines = file.readlines()
    print("".join(lines[:5]))  # Print the first 5 lines to verify

0.0;0.0;0.0;1.0;0.0056657250970602036;0.000894415017683059;-0.006727435160428286;0.0
0.0;0.0;0.0;1.0;-0.005258947145193815;-0.0030945499893277884;0.0071906400844454765;0.0
0.0;0.0;0.0;1.0;-0.0027163515333086252;-0.00023604999296367168;-0.0033335499465465546;0.0
0.0;0.0;0.0;1.0;0.015940947458148003;-0.000542099995072931;0.015590755268931389;0.0
0.0;0.0;0.0;1.0;-0.006049539893865585;-0.0007469850243069232;-0.005049725063145161;0.0



In [37]:
mesh_0 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0005.obj')
mesh_1 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\meshes/dancer_fr0006.obj')
deformed_mesh_0 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\output\Dancer\dq/000000.obj')

mesh_0.compute_vertex_normals()
mesh_1.compute_vertex_normals()
deformed_mesh_0.compute_vertex_normals()
deformed_mesh_0.paint_uniform_color([1, 0, 0])
mesh_1.paint_uniform_color([0, 1, 0])
o3d.visualization.draw_geometries([deformed_mesh_0])

In [29]:
mesh1 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0001.obj')
mesh2 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0002.obj')
mesh3 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0003.obj')
mesh4 = o3d.io.read_triangle_mesh(r'G:\VS2022Projects\arap-volume-tracking-main\data\Dancer/dancer_fr0004.obj')
mesh1.compute_vertex_normals()
mesh2.compute_vertex_normals()
mesh3.compute_vertex_normals()
mesh4.compute_vertex_normals()

o3d.visualization.draw_geometries([mesh1, mesh2, mesh3, mesh4])

In [24]:
import open3d as o3d
import numpy as np
spheres = []

# Define the radius of the spheres
sphere_radius = 0.01  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(loaded_transformed_centers.__len__()):
    point = loaded_transformed_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    mesh_sphere.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres += sphere

spheres = []

# Define the radius of the spheres
sphere_radius = 0.01  # Adjust the radius as needed
changed_sphere = o3d.geometry.TriangleMesh()
# Create a mesh sphere for each point and translate it to the point's location
count = 0
for i in range(loaded_centers.__len__()):
    point = loaded_centers[i]
    # Create a sphere mesh
    mesh_sphere = o3d.geometry.TriangleMesh.create_sphere(radius=sphere_radius)
    mesh_sphere.compute_vertex_normals()
    
    # Translate the sphere to the point's location
    mesh_sphere.translate(point)
    mesh_sphere.paint_uniform_color([0, 1, 0])  # Light grey color
    # Add the sphere to the list
    spheres.append(mesh_sphere)
print(spheres.__len__(), count)
# Combine all the spheres into one mesh
post_spheres1 = o3d.geometry.TriangleMesh()
for sphere in spheres:
    post_spheres1 += sphere
# Optionally, you can set the color of the spheres
#all_spheres.paint_uniform_color([0.7, 0.7, 0.7])  # Light grey color

# Draw the combined mesh
o3d.visualization.draw_geometries([post_spheres, mesh_1])

4000 0
4000 0
